In [30]:
import os

import pandas as pd
import numpy as np

from lifetimes.utils import summary_data_from_transaction_data

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

os.makedirs('data', exist_ok=True)

### **Fetching the Dataset**

In [31]:
df = pd.read_csv('data/prepared_data.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Status
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,Completed
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Completed
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,Completed
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Completed
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Completed


### **Simple Data Check**

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399573 entries, 0 to 399572
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    399573 non-null  object 
 1   StockCode    399573 non-null  object 
 2   Description  399573 non-null  object 
 3   Quantity     399573 non-null  int64  
 4   InvoiceDate  399573 non-null  object 
 5   UnitPrice    399573 non-null  float64
 6   CustomerID   399573 non-null  float64
 7   Country      399573 non-null  object 
 8   Status       399573 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 27.4+ MB


In [33]:
# Convert CustomerID to object type
df = df.astype({'CustomerID': 'object'})
# Convert InvoiceDate to datetime type
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%Y-%m-%d %H:%M:%S')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399573 entries, 0 to 399572
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    399573 non-null  object        
 1   StockCode    399573 non-null  object        
 2   Description  399573 non-null  object        
 3   Quantity     399573 non-null  int64         
 4   InvoiceDate  399573 non-null  datetime64[ns]
 5   UnitPrice    399573 non-null  float64       
 6   CustomerID   399573 non-null  object        
 7   Country      399573 non-null  object        
 8   Status       399573 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 27.4+ MB


In [34]:
# Statistical summary of numerical columns
df.select_dtypes(include=['int64', 'float64']).describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,399573.0,12.197999,250.088282,-80995.00,2.00,5.00,12.00,80995.0
UnitPrice,399573.0,2.905197,4.448902,0.03,1.25,1.95,3.75,649.5


In [35]:
# Statistical summary of categorical columns
df.describe(include='object').T

,count,unique,top,freq
InvoiceNo,399573,21781,576339,541
StockCode,399573,3674,85123A,2065
Description,399573,3863,WHITE HANGING HEART T-LIGHT HOLDER,2058
CustomerID,399573.0,4362.0,17841.0,7800.0
Country,399573,37,United Kingdom,356008
Status,399573,2,Completed,391068


### **Check transactions of each customer**

In [36]:
df_clv = df.copy()
df_clv['InvoiceDate_Day'] = pd.to_datetime(df_clv['InvoiceDate']).dt.date
df_clv = df_clv[['StockCode', 'CustomerID', 'InvoiceDate_Day', 'Quantity', 'UnitPrice']]
df_clv['Monetary'] = df_clv['UnitPrice'] * df_clv['Quantity']
df_clv.head()

,StockCode,CustomerID,InvoiceDate_Day,Quantity,UnitPrice,Monetary
0,85123A,17850.0,2010-12-01,6,2.55,15.30
1,71053,17850.0,2010-12-01,6,3.39,20.34
2,84406B,17850.0,2010-12-01,8,2.75,22.00
3,84029G,17850.0,2010-12-01,6,3.39,20.34
4,84029E,17850.0,2010-12-01,6,3.39,20.34


In [37]:
df_clv_stockcode = df_clv.groupby(['StockCode', 'CustomerID', 'InvoiceDate_Day'], as_index=False)['Monetary'].sum()
df_clv_stockcode = df_clv_stockcode[['StockCode', 'CustomerID', 'InvoiceDate_Day', 'Monetary']]

negative_or_zero_monetary = df_clv_stockcode[df_clv_stockcode['Monetary'] <= 0]
print(f"Number of rows with negative or zero Monetary values: {negative_or_zero_monetary.shape[0]}")

Number of rows with negative or zero Monetary values: 7881


In [38]:
df_clv_date = df_clv.groupby(['CustomerID', 'InvoiceDate_Day'], as_index=False)['Monetary'].sum()
df_clv_date = df_clv_date[['CustomerID', 'InvoiceDate_Day', 'Monetary']]

negative_or_zero_monetary = df_clv_date[df_clv_date['Monetary'] <= 0]
print(f"Number of rows with negative or zero Monetary values: {negative_or_zero_monetary.shape[0]}")

Number of rows with negative or zero Monetary values: 2539


1. Remove customers with transactions that sums up to negative

In [39]:
df_clv_negative = df_clv.groupby(['CustomerID'], as_index=False)['Monetary'].sum()
df_clv_negative = df_clv_negative[df_clv_negative['Monetary'] <= 1]

customer_ids_to_remove = df_clv_negative['CustomerID'].unique()
print(f"Number of customers with negative or zero total Monetary values: {len(customer_ids_to_remove)}")

df = df[~df['CustomerID'].isin(customer_ids_to_remove)]
df_clv = df_clv[~df_clv['CustomerID'].isin(customer_ids_to_remove)]
df_clv_date = df_clv_date[~df_clv_date['CustomerID'].isin(customer_ids_to_remove)]
df_clv_stockcode = df_clv_stockcode[~df_clv_stockcode['CustomerID'].isin(customer_ids_to_remove)]

Number of customers with negative or zero total Monetary values: 43


In [40]:
positive_monetary = df_clv_date[df_clv_date['Monetary'] > 1]
negative_or_zero_monetary = df_clv_date[df_clv_date['Monetary'] <= 1]
print(f"Number of rows with negative or zero Monetary values: {negative_or_zero_monetary.shape[0]}")
print("Statistical summary of negative or zero Monetary values:")
display(negative_or_zero_monetary.describe().T)
print("\n")
print(f"Number of rows with positive Monetary values: {positive_monetary.shape[0]}")
print("Statistical summary of positive Monetary values:")
display(positive_monetary.describe().T)

Number of rows with negative or zero Monetary values: 2498
Statistical summary of negative or zero Monetary values:


,count,mean,std,min,25%,50%,75%,max
Monetary,2498.0,-46.276257,211.44092,-7460.95,-30.75,-14.95,-6.7625,0.72




Number of rows with positive Monetary values: 16529
Statistical summary of positive Monetary values:


,count,mean,std,min,25%,50%,75%,max
Monetary,16529.0,507.231134,1246.415906,1.25,178.5,309.79,490.52,71601.44


In [41]:
negative_or_zero_monetary = negative_or_zero_monetary.sort_values(by='Monetary', ascending=True)
negative_or_zero_monetary.head()

,CustomerID,InvoiceDate_Day,Monetary
2032,12931.0,2011-08-31,-7460.95
16424,17450.0,2011-09-21,-3825.36
6101,14156.0,2011-02-25,-2548.80
12168,16029.0,2011-10-11,-1656.00
10533,15482.0,2011-11-04,-1652.80


In [42]:
df_clv_date[df_clv_date['CustomerID'] == 12931.0]

,CustomerID,InvoiceDate_Day,Monetary
2024,12931.0,2010-12-17,177.00
2025,12931.0,2011-02-15,1696.40
2026,12931.0,2011-03-15,62.50
2027,12931.0,2011-04-14,1488.00
2028,12931.0,2011-05-20,496.80
2029,12931.0,2011-08-04,19045.48
2030,12931.0,2011-08-11,9349.72
2031,12931.0,2011-08-30,214.80
2032,12931.0,2011-08-31,-7460.95
2033,12931.0,2011-10-03,1909.36


2. Remove negative daily transactions

In [43]:
print(f"Shape of df_clv_date before removing negative or zero Monetary values: {df_clv_date.shape}")
df_clv_date = df_clv_date[df_clv_date['Monetary'] > 1]
print(f"Shape of df_clv_date after removing negative or zero Monetary values: {df_clv_date.shape}")

Shape of df_clv_date before removing negative or zero Monetary values: (19027, 3)
Shape of df_clv_date after removing negative or zero Monetary values: (16529, 3)


In [44]:
df_clv_date['InvoiceDate_Day'] = pd.to_datetime(df_clv_date['InvoiceDate_Day'])
df_clv_date.head()

,CustomerID,InvoiceDate_Day,Monetary
1,12347.0,2010-12-07,711.79
2,12347.0,2011-01-26,475.39
3,12347.0,2011-04-07,636.25
4,12347.0,2011-06-09,382.52
5,12347.0,2011-08-02,584.91


### **Feature Engineering - RFMT Clustering**

In [45]:
df_rfmt = summary_data_from_transaction_data(
    df_clv_date,
    customer_id_col='CustomerID',
    datetime_col='InvoiceDate_Day',
    monetary_value_col='Monetary',
    observation_period_end=df_clv_date['InvoiceDate_Day'].max()
)

df_rfmt.head()

,frequency,recency,T,monetary_value
CustomerID,,,,
12347.0,6.0,365.0,367.0,599.701667
12348.0,3.0,283.0,358.0,261.480000
12349.0,0.0,0.0,18.0,0.000000
12350.0,0.0,0.0,310.0,0.000000
12352.0,5.0,260.0,296.0,193.782000


In [60]:
print(df_clv_date['InvoiceDate_Day'].max())

2011-12-09 00:00:00


In [61]:
df_clv_date[df_clv_date['CustomerID'] == 12347.0]

,CustomerID,InvoiceDate_Day,Monetary
1,12347.0,2010-12-07,711.79
2,12347.0,2011-01-26,475.39
3,12347.0,2011-04-07,636.25
4,12347.0,2011-06-09,382.52
5,12347.0,2011-08-02,584.91
6,12347.0,2011-10-31,1294.32
7,12347.0,2011-12-07,224.82


In [46]:
df_rfmt.rename(columns={
    'T': 'tenure',
    'monetary_value': 'monetary'}, inplace=True)

df_rfmt.head()

,frequency,recency,tenure,monetary
CustomerID,,,,
12347.0,6.0,365.0,367.0,599.701667
12348.0,3.0,283.0,358.0,261.480000
12349.0,0.0,0.0,18.0,0.000000
12350.0,0.0,0.0,310.0,0.000000
12352.0,5.0,260.0,296.0,193.782000


frequency 
- definition: number of repeat purchases the customer has made (one less than the total number of purchases)
- calculation: count of time periods the customer had a purchase in (if using days as units, then it’s the count of days the customer had a purchase on)

tenure
- definition: age of the customer in whatever time units chosen
- calculation: duration between a customer’s first purchase and the end of the period under study

recency
- definition: age of the customer when they made their most recent purchases
- calculation: duration between a customer’s first purchase and their latest purchase (if they have made only 1 purchase, the recency is 0)

monetary
- definition: average value of a given customer’s purchases
- calculation: sum of all a customer’s purchases divided by the total number of purchases

In [47]:
df_rfmt.describe().T

,count,mean,std,min,25%,50%,75%,max
frequency,4319.0,2.827043,5.880070,0.0,0.0,1.00,3.000,128.0000
recency,4319.0,130.806900,132.258900,0.0,0.0,93.00,252.000,373.0000
tenure,4319.0,222.977078,117.733412,0.0,113.0,248.00,326.000,373.0000
monetary,4319.0,259.105887,436.837723,0.0,0.0,174.67,356.545,9349.4772


In [48]:
# Filter customers with 0 recency, frequency, and monetary
zero_rfm_customers = df_rfmt[
    (df_rfmt['recency'] == 0) &
    (df_rfmt['frequency'] == 0) &
    (df_rfmt['monetary'] == 0)
]

print(f"Number of customers with 0 recency, frequency, and monetary: {zero_rfm_customers.shape[0]}")

Number of customers with 0 recency, frequency, and monetary: 1555


In [49]:
zero_rfm_customers.head()

,frequency,recency,tenure,monetary
CustomerID,,,,
12349.0,0.0,0.0,18.0,0.0
12350.0,0.0,0.0,310.0,0.0
12353.0,0.0,0.0,204.0,0.0
12354.0,0.0,0.0,232.0,0.0
12355.0,0.0,0.0,214.0,0.0


1. These are one time buyers so we remove them.

In [50]:
# Extract customer IDs (index) from zero_rfm_customers
zero_rfm_customer_ids = zero_rfm_customers.index

# Remove these customer IDs from df_clv_date, df, and df_rfmt
df_clv_date = df_clv_date[~df_clv_date['CustomerID'].isin(zero_rfm_customer_ids)]
df = df[~df['CustomerID'].isin(zero_rfm_customer_ids)]
df_rfmt = df_rfmt[~df_rfmt.index.isin(zero_rfm_customer_ids)]

In [51]:
df_rfmt.describe().T

,count,mean,std,min,25%,50%,75%,max
frequency,2764.0,4.417511,6.856056,1.000000,1.000,3.00,5.000,128.0000
recency,2764.0,204.397612,110.859528,1.000000,107.000,212.00,302.000,373.0000
tenure,2764.0,260.641100,101.000339,9.000000,198.000,284.00,358.000,373.0000
monetary,2764.0,404.876384,489.063612,6.783333,186.065,307.53,453.625,9349.4772


In [52]:
df_rfmt.to_csv('data/customer_rfmt_final.csv')
df_clv_date.to_csv('data/customer_rfmt_data.csv', index=False)
df.to_csv('data/prepared_data_feature_engg.csv', index=False)

### **Feature Engineering - Product Description**

1. Basic Filtering

In [53]:
df['Description_Modified_00'] = df['Description'].str.strip()
df['Description_Modified_00'] = df['Description_Modified_00'].str.replace(r'[,.]', ' ', regex=True).str.strip()
df['Description_Modified_00'] = df['Description_Modified_00'].str.replace(r'CHILDS', 'CHILD', regex=True).str.strip()
df['Description_Modified_00'] = df['Description_Modified_00'].str.replace(r'CHILDRENS', 'CHILDREN', regex=True).str.strip()
df['Description_Modified_00'] = df['Description_Modified_00'].str.replace(r"CHILDREN'S", 'CHILDREN', regex=True).str.strip()
df['Description_Modified_00'] = df['Description_Modified_00'].str.replace(r"CHILD'S", 'CHILD', regex=True).str.strip()
df['Description_Modified_00'] = df['Description_Modified_00'].str.replace(r"SET OF", 'SET ', regex=True).str.strip()
df['Description_Modified_00'] = df['Description_Modified_00'].str.replace(r"PACK OF", 'PACK ', regex=True).str.strip()
df['Description_Modified_00'] = df['Description_Modified_00'].str.replace(r"BOX OF", 'BOX ', regex=True).str.strip()
df['Description_Modified_00'] = df['Description_Modified_00'].str.replace(r"BUNDLE OF", 'BUNDLE ', regex=True).str.strip()
df['Description_Modified_00'] = df['Description_Modified_00'].str.replace(r"(\d+)CMX", r"\1X", regex=True).str.strip()
df['Description_Modified_00'] = df['Description_Modified_00'].str.split().str.join(' ').str.strip()
df[['Description', 'Description_Modified_00']].head()

,Description,Description_Modified_00
0,WHITE HANGING HEART T-LIGHT HOLDER,WHITE HANGING HEART T-LIGHT HOLDER
1,WHITE METAL LANTERN,WHITE METAL LANTERN
2,CREAM CUPID HEARTS COAT HANGER,CREAM CUPID HEARTS COAT HANGER
3,KNITTED UNION FLAG HOT WATER BOTTLE,KNITTED UNION FLAG HOT WATER BOTTLE
4,RED WOOLLY HOTTIE WHITE HEART,RED WOOLLY HOTTIE WHITE HEART


2. Fixing Multiple Descriptions to 1 StockCode

In [54]:
# Group by StockCode and aggregate unique Description_Modified_00
stock_description_details = df.groupby('StockCode')['Description_Modified_00'].unique()
# Filter StockCodes with more than 1 unique Description_Modified_00
stock_with_multiple_descriptions = stock_description_details[stock_description_details.apply(len) > 1]

print(f"Number of StockCodes with more than 1 unique Description_Modified_00: {stock_with_multiple_descriptions.shape[0]}")

Number of StockCodes with more than 1 unique Description_Modified_00: 185


In [55]:
# Create a function to determine the most appropriate description
def resolve_description(stockcode_group):
    # Count the frequency of each Description_Modified_00
    description_counts = stockcode_group['Description_Modified_00'].value_counts()

    # Get the most frequent description(s)
    max_count = description_counts.max()
    most_frequent = description_counts[description_counts == max_count].index.tolist()

    # If there's only one most frequent description, return it
    if len(most_frequent) == 1:
        return most_frequent[0]

    # If there are ties, pick the one with no numbers or punctuations
    no_numbers_or_punctuations = [
        desc for desc in most_frequent if not any(char.isdigit() or not char.isalnum() for char in desc.replace(" ", ""))
    ]
    if no_numbers_or_punctuations:
        return no_numbers_or_punctuations[0]

    # If still tied, pick one randomly
    return np.random.choice(most_frequent)

# Apply the function to StockCodes with multiple unique descriptions
df['Description_Modified_01'] = df['Description_Modified_00']  # Default to Description_Modified_00
stock_with_multiple_descriptions = df.groupby('StockCode').filter(
    lambda x: x['Description_Modified_00'].nunique() > 1
)
resolved_descriptions = stock_with_multiple_descriptions.groupby('StockCode').apply(resolve_description)

# Map the resolved descriptions back to the dataframe
df.loc[df['StockCode'].isin(resolved_descriptions.index), 'Description_Modified_01'] = df['StockCode'].map(resolved_descriptions)

# Verify the changes
df[['StockCode', 'Description_Modified_00', 'Description_Modified_01']].head()

,StockCode,Description_Modified_00,Description_Modified_01
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART,RED WOOLLY HOTTIE WHITE HEART


3. Fixing Multiple StockCodes to 1 Unique Description

In [56]:
# Group by Description and aggregate unique StockCode
description_stockcode_details = df.groupby('Description_Modified_01')['StockCode'].unique()

# Filter descriptions with more than 1 unique StockCode
descriptions_with_multiple_stockcodes = description_stockcode_details[description_stockcode_details.apply(len) > 1]

print(f"Number of descriptions with more than 1 unique StockCode: {descriptions_with_multiple_stockcodes.shape[0]}")

Number of descriptions with more than 1 unique StockCode: 28


In [57]:
import numpy as np

# Create a function to determine the most appropriate StockCode
def resolve_stockcode(description_group):
    # Count the frequency of each StockCode
    stockcode_counts = description_group['StockCode'].value_counts()

    # Get the most frequent StockCode(s)
    max_count = stockcode_counts.max()
    most_frequent = stockcode_counts[stockcode_counts == max_count].index.tolist()

    # If there's only one most frequent StockCode, return it
    if len(most_frequent) == 1:
        return most_frequent[0]

    # If there are ties, pick the StockCode with only 1 unique Description_Modified_01
    stockcode_with_one_description = [
        stock for stock in most_frequent
        if description_group[description_group['StockCode'] == stock]['Description_Modified_01'].nunique() == 1
    ]
    if stockcode_with_one_description:
        return stockcode_with_one_description[0]

    # If still tied, pick one randomly
    return np.random.choice(most_frequent)

# Apply the function to Descriptions with multiple unique StockCodes
df['StockCode_Modified'] = df['StockCode']  # Default to StockCode
descriptions_with_multiple_stockcodes = df.groupby('Description_Modified_01').filter(
    lambda x: x['StockCode'].nunique() > 1
)
resolved_stockcodes = descriptions_with_multiple_stockcodes.groupby('Description_Modified_01').apply(resolve_stockcode)

# Map the resolved StockCodes back to the dataframe
df.loc[df['Description_Modified_01'].isin(resolved_stockcodes.index), 'StockCode_Modified'] = df['Description_Modified_01'].map(resolved_stockcodes)

# Verify the changes
df[['Description_Modified_01', 'StockCode', 'StockCode_Modified']].head()

,Description_Modified_01,StockCode,StockCode_Modified
0,WHITE HANGING HEART T-LIGHT HOLDER,85123A,85123A
1,WHITE METAL LANTERN,71053,71053
2,CREAM CUPID HEARTS COAT HANGER,84406B,84406B
3,KNITTED UNION FLAG HOT WATER BOTTLE,84029G,84029G
4,RED WOOLLY HOTTIE WHITE HEART,84029E,84029E


4. Removing Numbers

In [58]:
# Remove leading and trailing white spaces
df['Description_Modified_02'] = df['Description_Modified_01'].str.strip()
# Remove " (number) " for every " (number) "
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'\s\d+\s', ' ', regex=True).str.strip()
# Remove "S/(number) " for every "S/(number) "
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'S/\d+\s', ' ', regex=True).str.strip()
# Remove "/(number)" for every "/(number)"
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'/\d+', '', regex=True).str.strip()
# Remove "(number)'S" for every "(number)'S"
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r"\d+'S", '', regex=True).str.strip()
# Remove "(number)CMX" for every "(number)CMX"
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'\d+CMX', '', regex=True).str.strip()
# Remove "X(number)CM" for every "X(number)CM"
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'X\d+CM', '', regex=True).str.strip()
# Remove "(number)CM" for every "(number)CM"
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'\d+CM', '', regex=True).str.strip()
# Remove "(number)X" for every "(number)X"
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'\d+X', '', regex=True).str.strip()
# Remove "(number)PC" for every "(number)PC"
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'\d+PC', '', regex=True).str.strip()
# Remove "(A)(number)" for every "(A)(number)"
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'A\d+', '', regex=True).str.strip()
# Remove " (number)" for every " (number)" and there is no space after the number
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'\s\d+(?!\s)', '', regex=True).str.strip()
# Change "(number)(")" to " "
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'\d+\(', ' ', regex=True)
# Remove "(number) " for every "(number) " and there is no space before the number
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'(?<!\s)\d+\s', '', regex=True).str.strip()
# Remove "(number)" in every "(NECKL)(number)"
df['Description_Modified_02'] = df['Description_Modified_02'].str.replace(r'NECKL\d+', 'NECKL', regex=True).str.strip()
# Remove extra spaces
df['Description_Modified_02'] = df['Description_Modified_02'].str.split().str.join(' ').str.strip()
# Verify the changes
df[['Description', 'Description_Modified_01', 'Description_Modified_02']].head()

,Description,Description_Modified_01,Description_Modified_02
0,WHITE HANGING HEART T-LIGHT HOLDER,WHITE HANGING HEART T-LIGHT HOLDER,WHITE HANGING HEART T-LIGHT HOLDER
1,WHITE METAL LANTERN,WHITE METAL LANTERN,WHITE METAL LANTERN
2,CREAM CUPID HEARTS COAT HANGER,CREAM CUPID HEARTS COAT HANGER,CREAM CUPID HEARTS COAT HANGER
3,KNITTED UNION FLAG HOT WATER BOTTLE,KNITTED UNION FLAG HOT WATER BOTTLE,KNITTED UNION FLAG HOT WATER BOTTLE
4,RED WOOLLY HOTTIE WHITE HEART,RED WOOLLY HOTTIE WHITE HEART,RED WOOLLY HOTTIE WHITE HEART


In [59]:
df.to_csv('data/prepared_data_descriptions.csv', index=False)